In [47]:
import cv2
import mediapipe as mp
import numpy as np
import time #Dùng để tính thời gian cooldown cho các cảnh báo

#Nhập khẩu Spicy để tính Euclidean Distance
from scipy.spatial import distance as dist

# Nhập khẩu Task API (Dành cho Landmarker)
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Nhập khẩu các lớp Image và Định dạng (Được đặt ở cấp cao trong MP mới)
from mediapipe import ImageFormat
from mediapipe import Image as MpImage


In [49]:
#--- HÀM TÍNH EAR (Ear Aspect Ratio) ---

def eye_aspect_ratio(eye): 
    # Tính khoảng cách Euclidean giữa hai tập hợp tọa độ thẳng đứng của mắt
    # p2 - p6 và p3 - p5
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # Tính khoảng cách Euclidean giữa hai điểm ngang của mắt
    #p1 - p4 (Khoe mat)
    C = dist.euclidean(eye[0], eye[3])

    # Tính EAR
    ear = (A + B) / (2.0 * C)
    return ear


In [50]:
# --- Ham tinh MAR (Mouth Aspect Ratio) ---
def mouth_aspect_ratio(mouth):
    # Sử dụng các điểm 2, 6 (thẳng đứng) và 1, 4 (nằm ngang) cho MAR
    A = dist.euclidean(mouth[2], mouth[6])  # Khoảng cách dọc (chieu cao miệng)
    C = dist.euclidean(mouth[1], mouth[4])  # Khoảng cách ngang (chieu rong miệng)
    mar = A / C
    return mar

In [ ]:
# --- Các hằng số của Head Pose ---
#1. Các landmark 2D/3D tương ứng cho Head Pose (từ MediaPipe)
# Các điểm landmark được dùng làm mốc
# 33: Cánh mũi trái, 263: Cánh mũi phải, 1: Mũi, 61: Mép miệng trái, 291: Mép miệng phải, 199: Cằm

HEAD_POSE_INDICES = [33, 263, 1, 61, 291]

#2. Các điểm 3D (Object Points) tham chiếu của khuôn mặt mẫu (đơn vị: mm hoặc pixel)
# Toạ độ là cố định dùng so sánh để biết mat có bị nghiêng hay không

MODEL_3D_POINTS = np.array([
    (0.0, 0.0, 0.0),                # Nose tip: 1
    (-225.0, 170.0, -135.0),        # Cánh mũi trái (Left Eye Corner: 33)
    (225.0, 170.0, -135.0),         # Right eye right corner
    (-150.0, -150.0, -125.0),       # Khóe miệng trái (Left Mouth Corner: 61)
    (150.0, -150.0, -125.0)         # Khóe miệng phải (Right Mouth Corner: 291)
], dtype=np.float32)

In [51]:
# --- Khoi tao model MediaPipe Face Mesh ---

#1. Duong dan toi file .taks
model_path = "face_landmarker.task"

#2. Tao doi tuong BaseOptions va FaceLandmarkerOptions
base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.FaceLandmarkerOptions(
    base_options = base_options,
    output_face_blendshapes = False,
    output_facial_transformation_matrixes = False,
    #Che do chay: Streaming cho camera (thuc hien qua call back)
    running_mode = vision.RunningMode.IMAGE,
    num_faces = 1 # Phat hien 1 khuon mat
)

#3. Khoi tao FaceLandmarker
landmarker = vision.FaceLandmarker.create_from_options(options)

#Giu nguyen cac chi muc va nguong da dinh nghia
RIGHT_EYE_INDICES = [33, 160, 158, 133, 153, 144] #Các điểm mắt phải
LEFT_EYE_INDICES = [362, 385, 387, 263, 373, 380] #Các điểm mắt trái
MOUTH_INDICES = [61, 291, 0, 17, 14, 87, 317, 324]#Các điểm miệng
EAR_THRESHOLD = 0.2 #Giá trị ngưỡng EAR
MAR_THRESHOLD = 1 #Giá trị ngưỡng MAR
CONSECUTIVE_FRAMES = 30 #Số khung hình liên tiếp = 30
COOL_DOWN_PERIOD = 30 #Thời gian nghỉ giữa các cảnh báo 
LAST_ALERT_TIME_DROWSINESS = 0.0 # Tách biến cho từng loại cảnh báo
LAST_ALERT_TIME_YAWN = 0.0
ALERT_ACTIVE = False

COUNTER = 0

cap = cv2.VideoCapture(0)

In [ ]:
def log_alert_terminal(alert_type):
    """
    Xử lý logic cooldown và in cảnh báo ra terminal.
    Trả về True nếu cảnh báo được in, False nếu đang trong cooldown.
    """

    global LAST_ALERT_TIME_DROWSINESS, LAST_ALERT_TIME_YAWN
    current_time = time.time()


    # Văn bản hiển thị
    if alert_type == "DROWSINESS":
        last_alert_time = LAST_ALERT_TIME_DROWSINESS
    else: # YAWN
        last_alert_time = LAST_ALERT_TIME_YAWN

    # Kiểm tra cooldown
    if(current_time - last_alert_time) > COOL_DOWN_PERIOD:
        #1. Gửi cảnh báo ra terminal
        print("=========================================")
        print(f"!!! ALERT: {alert_type} ACTIVATED!!!")
        print(f"Alert sent at: {time.ctime(current_time)}")
        print("=========================================")

        #2. Cập nhật thời gian cảnh báo cuối cùng
        if alert_type == "DROWSINESS":
            LAST_ALERT_TIME_DROWSINESS = current_time
        else: #YAWN
            LAST_ALERT_TIME_YAWN = current_time

        return True #Đã in cảnh báo
    
    return False #Đang trong thời gian cooldown

In [53]:
# --- Vong lap xu ly frame tu camera ---
while cap.isOpened():

    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        break

    #1. Tien xu Ly frame va chuyen doi sang doi tuong MediaPipe Image
    image = cv2.flip(image,1)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Create a MediaPipe Tasks Vision Image with explicit format
    mp_image = MpImage(image_format=ImageFormat.SRGB, data=rgb_image)

    #2. Xu ly landmark
    face_landmarker_result = landmarker.detect(mp_image)

    #3. Hau xu ly va trich xuat
    h, w, c = image.shape

    # Kiem tra xem co phat hien khuon mat khong
    if face_landmarker_result.face_landmarks:
        #Lay landmark khuon mat dau tien
        landmarks_list = face_landmarker_result.face_landmarks[0]
        lanmarks_points = []

        #Chuyen doi toa do normalized (0-1) sang toa do pixel
        for landmark in landmarks_list:
            x = int(landmark.x * w)
            y = int(landmark.y * h)
            lanmarks_points.append((x,y))

        # --- TÍNH TOÁN EAR và MAR (Code tính toán giữ nguyên) ---
        left_eye_coords = [lanmarks_points[i] for i in LEFT_EYE_INDICES]
        right_eye_coords = [lanmarks_points[i] for i in RIGHT_EYE_INDICES]
        mouth_coords = [lanmarks_points[i] for i in MOUTH_INDICES]

        avg_ear = (eye_aspect_ratio(left_eye_coords) + eye_aspect_ratio(right_eye_coords)) / 2.0
        mar = mouth_aspect_ratio(mouth_coords)

        # --- LOGIC DROWINESS DETECTION (Giữ nguyên) ---
        # ... (Hiển thị EAR/MAR và logic cảnh báo)
        cv2.putText(image, f"EAR: {avg_ear:.2f}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(image, f"MAR {mar:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        
        # Biến cờ (Flag) để theo dõi trạng thái cảnh báo đang hoạt động
        drowsy_flag = False
        yawn_flag = False
        
        #Logic cảnh báo ngủ gật (DROWSINESS ALERT! & YAWN ALERT!)

        #DROWSINESS ALERT
        if avg_ear < EAR_THRESHOLD:
            COUNTER += 1

            if COUNTER >= CONSECUTIVE_FRAMES:
                drowsy_flag = True
                
                #1. Hiển thị cảnh báo trên giao diện (Liên tục trên Open CV)
                cv2.putText(image, "DROWSINESS ALERT!", (10, h - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
                    
                #2. Logic xử lý Terminal (Cooldown)
                log_alert_terminal("DROWSINESS")
        else: 
            COUNTER = 0
                    
        #YAWN ALERT
        if mar > MAR_THRESHOLD and not drowsy_flag: # Ưu tiên DROWSINESS
            yawn_flag = True

            #1. Hiển thị cảnh báo trên giao diện (Liên tục trên Open CV)
            cv2.putText(image, "YAWN ALERT!", (10, h - 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 3)
            
            #2. Xử lý Terminal (Cooldown)
            log_alert_terminal("YAWN")

    # Hien thi frame
    cv2.imshow("Drowsiness Detector", image)

    if cv2.waitKey(5) & 0xFF == ord('q'): #Bam "q" de thoat
        break

# Giai phong bo nho
cap.release()
cv2.destroyAllWindows()

!!! ALERT: DROWSINESS ACTIVATED!!!
Alert sent at: Tue Dec  9 22:15:24 2025
!!! ALERT: DROWSINESS ACTIVATED!!!
Alert sent at: Tue Dec  9 22:15:55 2025
!!! ALERT: DROWSINESS ACTIVATED!!!
Alert sent at: Tue Dec  9 22:15:55 2025
!!! ALERT: YAWN ACTIVATED!!!
Alert sent at: Tue Dec  9 22:16:05 2025
!!! ALERT: YAWN ACTIVATED!!!
Alert sent at: Tue Dec  9 22:16:05 2025
!!! ALERT: DROWSINESS ACTIVATED!!!
Alert sent at: Tue Dec  9 22:16:25 2025
!!! ALERT: DROWSINESS ACTIVATED!!!
Alert sent at: Tue Dec  9 22:16:25 2025
!!! ALERT: YAWN ACTIVATED!!!
Alert sent at: Tue Dec  9 22:16:39 2025
!!! ALERT: YAWN ACTIVATED!!!
Alert sent at: Tue Dec  9 22:16:39 2025
!!! ALERT: DROWSINESS ACTIVATED!!!
Alert sent at: Tue Dec  9 22:16:55 2025
!!! ALERT: DROWSINESS ACTIVATED!!!
Alert sent at: Tue Dec  9 22:16:55 2025
!!! ALERT: YAWN ACTIVATED!!!
Alert sent at: Tue Dec  9 22:17:09 2025
!!! ALERT: YAWN ACTIVATED!!!
Alert sent at: Tue Dec  9 22:17:09 2025
!!! ALERT: DROWSINESS ACTIVATED!!!
Alert sent at: Tue Dec  9 

C:\Users\Admin\AppData\Local\Temp\ipykernel_10660\3540102838.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  mar = A / C
